In [ ]:
import sys
import gplately
import numpy as np
import gplately.pygplates as pygplates
from gplately import ptt
import glob, os
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from plate_model_manager import PlateModelManager

# directory to the aspect Lab
ASPECT_LAB_DIR = os.environ['ASPECT_LAB_DIR']
# import utilities in subdirectiory
sys.path.append(os.path.join(ASPECT_LAB_DIR, 'utilities', "python_scripts"))
import Utilities


anchor_plate_id = 0

### PlateModelManager

The **PlateModelManager** class can be used to download the plate reconstruction model files via Internet. The files will be fetched and saved into local folders. Users can use the member functions of PlateModel class to retrieve the local absolute path of the files. For example, the get_rotation_model() will return the local path to the rotation file(s).

The **PlateReconstruction** class can be used to reconstruct tectonic plates and calculate subduction convergence stats.

The **PlotTopologies** class can be used to plot reconstruction maps.

In [ ]:
%%capture cap
# Use PlateModelManager to download and manage files of the plate reconstruction models
pm_manager = PlateModelManager()
plate_model = pm_manager.get_model("Muller2019", data_dir="plate-model-repo")

model = gplately.PlateReconstruction(plate_model.get_rotation_model(), 
                                     plate_model.get_topologies(), 
                                     plate_model.get_static_polygons(),
                                     anchor_plate_id=anchor_plate_id)
gplot = gplately.plot.PlotTopologies(model, 
                                     plate_model.get_layer('Coastlines'), 
                                     plate_model.get_layer('ContinentalPolygons'), 
                                     plate_model.get_layer('COBs'))

### Query the subduction data

In [ ]:
"""Assign an reconstruction time and filter the subduction date accordingly

# Calculate subduction convergence stats with GPlately
# Col. 0 - longitude of sampled trench point
# Col. 1 - latitude of sampled trench point
# Col. 2 - subducting convergence (relative to trench) velocity magnitude (in cm/yr)
# Col. 3 - subducting convergence velocity obliquity angle (angle between trench normal vector and convergence velocity vector)
# Col. 4 - trench absolute (relative to anchor plate) velocity magnitude (in cm/yr)
# Col. 5 - trench absolute velocity obliquity angle (angle between trench normal vector and trench absolute velocity vector)
# Col. 6 - length of arc segment (in degrees) that current point is on
# Col. 7 - trench normal azimuth angle (clockwise starting at North, ie, 0 to 360 degrees) at current point
# Col. 8 - subducting plate ID
# Col. 9 - trench plate ID
"""
reconstruction_time = 0 # Ma, must be integer, otherwise the query seafloor age shoots error

all_columns = ['lon', 'lat', 'conv_rate', 'conv_angle', 'trench_velocity', 'trench_velocity_angle', 'arc_length',
                                     'trench_azimuth_angle', 'subducting_pid', 'trench_pid']

subduction_data = model.tessellate_subduction_zones(reconstruction_time, 
                                                    # tessellation_threshold_radians=0.01, 
                                                    anchor_plate_id=anchor_plate_id,
                                                    ignore_warnings=True)

# get all the trench ids
temp = [row[9] for row in subduction_data]
trench_ids = sorted(set(temp))

### Query the sea floor age data

In [ ]:
age_grid_raster = gplately.Raster(
    data=plate_model.get_raster("AgeGrids",reconstruction_time),
    plate_reconstruction=model,
    extent=[-180, 180, -90, 90],
)

### Define the function to calculate and filter subduction zone stats for South America

In [ ]:
# We are only interested in the latitude -55 and 5 in this notebook. 
# lat_samples = np.arange(-55,5,0.1)

def get_one_subduction_by_trench_id(subduction_data, trench_id):
    '''
    get one subduction data from the whole dataset at one reconstruction time
    Inputs:
        subduction_data - global dataset of subduction zones at a reconstruction time
        trench_id - id of one subduction zone
    '''
    # ret is a list, each component is a list of 10 entries, corresponding to the 10 columns in the final panda frame 
    ret=[]
    for row in subduction_data:
        # only keep the data for South America subduction zone
        # row[8] is the id of the subduction, while row[9] is the id of the trench
        # here if we don't sort row[8], the result is identical
        # not sure what row[9] > 200000 suppose to mean
        # if row[8] in [902,908,919,904,911,985,224,802] and (row[9]==201 or row[9]>200000): # code copied from Simon's original notebook
        #  if True and (row[9]==201 or row[9]>200000): # select entry with row[9] and also append big value
        if True and (row[9]==trench_id): # only select entry with row 9
            ret.append(row)
    ret.sort(key=lambda row: row[1])
    one_subduction_data = pd.DataFrame(ret, columns=all_columns)
    
    # one_subduction_data = ret_df[(ret_df.lat>lat_samples.min()) & (ret_df.lat<lat_samples.max())]
    return one_subduction_data

def sample_subduction_center(one_subduction_data):
    '''
    '''
    one_subduction_data.arc_length
    pass

def plot_one_subduction_data(ax, one_subduction_data, **kwargs):
    '''
    plot one subduction zone in a global projection
    Inputs:
        ax - plot axis
        one_subduction_data - dataset of one subduction zone
    '''
    attribute = kwargs.get("attribute", "conv_rate") # the attribute to plot
    # Latitudes and longitudes of points along trench segments
    subduction_lon = one_subduction_data.lon
    subduction_lat = one_subduction_data.lat

    # get the data to plot with
    data_attr = getattr(one_subduction_data, attribute) 

    # figure configurations
    vmin = None; vmax = None
    if attribute == "conv_rate":
        vmin = 0.0
        vmax = 20.0
        cmap="inferno_r"
    elif attribute == 'trench_velocity':
        vmin = -5.0
        vmax = 5.0
        cmap="viridis_r"
    else:
        raise NotImplementedError()
    
    # plot
    cb=ax.scatter(subduction_lon,subduction_lat, marker=".", s=5, c=data_attr, transform=ccrs.PlateCarree(), cmap=cmap, vmin=vmin, vmax=vmax)

    return cb

def ResampleSubduction(one_subduction_data, arc_length_edge, arc_length_resample_section, **kwargs):
    '''
    Resample data of a subduction to a couple of query points. 
    This is used to extract sample points of an otherwise dense subduction zone and make plots of subduction zone properties
    Inputs:
        one_subduction_data - panda data of one subduction zone
        arc_length_edge - edge of the subduction zone, filtered out
        arc_length_resample_section - resampled section of arc length
    kwargs:
        indent - indent for outputs
    '''
    # initiate
    indent = kwargs.get("indent", 0) # the default is to not indent output
    log_output_contents = ""
    data_len = len(one_subduction_data)
    
    # get the arc_length_sums
    arc_lengths = one_subduction_data['arc_length']
    arc_length_sums = np.zeros(data_len)
    arc_length_sums[0] = arc_lengths[0]
    for i in range(1, data_len):
        arc_length_sums[i] = arc_length_sums[i-1] + arc_lengths[i]

    # get the resampled arc sums
    # The resampling procedure is initiated at the center and propogated with
    # an interval of arc_length_resample_section towards both end.
    # It ends within arc_length_edge distance of the end point. 
    temp = []
    if arc_length_sums[-1] > 2 * arc_length_edge:
        temp.append(arc_length_sums[-1] / 2.0)
    i = 1
    arc_length_sum_temp = arc_length_sums[-1] / 2.0 - arc_length_resample_section
    arc_length_sum_temp1 = arc_length_sums[-1] / 2.0 + arc_length_resample_section
    while arc_length_sum_temp > arc_length_edge:
        temp.append(arc_length_sum_temp)
        temp.append(arc_length_sum_temp1)
        arc_length_sum_temp -= arc_length_resample_section
        arc_length_sum_temp1 += arc_length_resample_section
    arc_length_sums_resampled = sorted(temp)

    # resample the properties of the subduction by interpolation 
    one_subduction_data_resampled = pd.DataFrame(columns=all_columns)
    i_sbd_re = 0
    for arc_length_sum_resampled in arc_length_sums_resampled:
        for i in range(len(arc_length_sums)-1):
            if (arc_length_sums[i] <= arc_length_sum_resampled) and (arc_length_sum_resampled < arc_length_sums[i+1]):
                fraction = (arc_length_sum_resampled - arc_length_sums[i]) /  (arc_length_sums[i+1] - arc_length_sums[i])
                row_temp = fraction * one_subduction_data.iloc[i] + (1. - fraction) * one_subduction_data.iloc[i+1]
                # longitude and latitude are interpolated using a special method
                row_temp.lon, row_temp.lat = Utilities.map_mid_point(one_subduction_data.iloc[i].lon, one_subduction_data.iloc[i].lat,\
                                                                     one_subduction_data.iloc[i+1].lon, one_subduction_data.iloc[i+1].lat, fraction)
                log_output_contents += "%s%d th resampled point: (%.2f, %.2f)\n" % (" "*indent, i_sbd_re, row_temp.lon, row_temp.lat) # record the resampled point

                one_subduction_data_resampled = pd.concat([one_subduction_data_resampled,  pd.DataFrame([row_temp])], ignore_index=True)
        i_sbd_re += 1

    return one_subduction_data_resampled, log_output_contents

### Plot one trench

In [ ]:
# assign the trench id for south america
trench_id = 201

assert(trench_id in trench_ids)

one_subduction_data = get_one_subduction_by_trench_id(subduction_data, trench_id)

# plot the reconstructed zone

fig = plt.figure(figsize=(10,6), dpi=100)
ax = fig.add_subplot(111, projection=ccrs.Mollweide(central_longitude = 0))

gl=ax.gridlines(color='0.7',linestyle='--', xlocs=np.arange(-180,180,15), ylocs=np.arange(-90,90,15))
gl.left_labels = True

plt.title(f'{reconstruction_time} Ma')
# you may change the extent to global to see the sample points in a world map.
ax.set_global()
# ax.set_extent([-80,0,-70,0])

gplot.time = reconstruction_time
gplot.plot_coastlines(ax, color='grey')

cb = plot_one_subduction_data(ax, one_subduction_data)

cbar = plt.colorbar(cb)
cbar.ax.get_yaxis().labelpad = 15
cbar.ax.set_ylabel('Convergence Velocity Magnitude (in cm/yr)', rotation=90)
plt.show()

In [ ]:
arc_length_edge = 5.0  # by degree
arc_length_resample_section = 10.0

one_subduction_data_resampled = ResampleSubduction(one_subduction_data, arc_length_edge, arc_length_resample_section)
print(one_subduction_data_resampled)

# plot the reconstructed zone

fig = plt.figure(figsize=(10,6), dpi=100)
ax = fig.add_subplot(111, projection=ccrs.Mollweide(central_longitude = 0))

gl=ax.gridlines(color='0.7',linestyle='--', xlocs=np.arange(-180,180,15), ylocs=np.arange(-90,90,15))
gl.left_labels = True

plt.title(f'{reconstruction_time} Ma')
# you may change the extent to global to see the sample points in a world map.
ax.set_global()
# ax.set_extent([-80,0,-70,0])

gplot.time = reconstruction_time
gplot.plot_coastlines(ax, color='grey')

cb=ax.scatter(one_subduction_data_resampled.lon, one_subduction_data_resampled.lat, marker=".", s=15, c='r', transform=ccrs.PlateCarree())

cbar = plt.colorbar(cb)
cbar.ax.get_yaxis().labelpad = 15
cbar.ax.set_ylabel('Convergence Velocity Magnitude (in cm/yr)', rotation=90)
plt.show()

### Plot all trenches

In [ ]:
attribute = "trench_velocity"
# resample points by arc length
arc_length_edge = 5.0  # by degree
arc_length_resample_section = 10.0
log_output = os.path.join(ASPECT_LAB_DIR, "dtemp", "resampled_subduction_t%.2f.txt" % reconstruction_time) # file to output log
log_output_contents = ""

log_output_contents += "arc_length_edge = %.2f, arc_length_resample_section = %.2f" % (arc_length_edge, arc_length_resample_section)

# plot the reconstructed zone

fig = plt.figure(figsize=(10,6), dpi=100)
ax = fig.add_subplot(111, projection=ccrs.Mollweide(central_longitude = 180))

gl=ax.gridlines(color='0.7',linestyle='--', xlocs=np.arange(-180,180,15), ylocs=np.arange(-90,90,15))
gl.left_labels = True

plt.title(f'{reconstruction_time} Ma')
# you may change the extent to global to see the sample points in a world map.
ax.set_global()
# ax.set_extent([-80,0,-70,0])

gplot.time = reconstruction_time
gplot.plot_coastlines(ax, color='grey')

# assign the trench id for south america
subduction_data_resampled_list = [] # assign the resampled data list
total_points = 0
for i in range(len(trench_ids)):
    trench_id = trench_ids[i]
    one_subduction_data = get_one_subduction_by_trench_id(subduction_data, trench_id)
    log_output_contents += "%d th arc\n" % i
    log_output_contents += "start (%.2f, %.2f)\n" % (one_subduction_data.iloc[0].lon, one_subduction_data.iloc[0].lat)
    log_output_contents += "end (%.2f, %.2f)\n" % (one_subduction_data.iloc[-1].lon, one_subduction_data.iloc[-1].lat)
    # log_output_contents += "length in degree: %.2f\n" % (one_subduction_data["arc_length"][len(one_subduction_data)-1])
    # print("one_subduction_data.lon: \n", one_subduction_data.lon) # debug
    # print("one_subduction_data.lat: \n", one_subduction_data.lat)
    # resample the dense subduction zones
    one_subduction_data_resampled, log_output_contents_temp = ResampleSubduction(one_subduction_data, arc_length_edge, arc_length_resample_section, indent=4)
    log_output_contents += log_output_contents_temp
    # print("one_subduction_data_resampled.lon: \n", one_subduction_data_resampled.lon) # debug
    # print("one_subduction_data_resampled.lat: \n", one_subduction_data_resampled.lat)
    total_points += len(one_subduction_data_resampled) # record the number of total points
    subduction_data_resampled_list.append(one_subduction_data_resampled)
    cb = plot_one_subduction_data(ax, one_subduction_data, attribute=attribute)
    ax.scatter(one_subduction_data_resampled.lon, one_subduction_data_resampled.lat, marker=".", s=15, c='r', transform=ccrs.PlateCarree())
log_output_contents += "Total interpolated points: %d" % total_points
    
# write log output file
if log_output is not None:
    with open(log_output, 'w') as fout:
        fout.write(log_output_contents)
    print("log file: %s" % log_output)

# set plot options
cbar = plt.colorbar(cb)
cbar.ax.get_yaxis().labelpad = 15
label = None
if attribute == "conv_rate":
    label = 'Convergence Velocity Magnitude (in cm/yr)'
elif attribute == "trench_velocity":
    label = 'Trench Velocity Magnitude (in cm/yr)'
else:
    raise NotImplementedError()
cbar.ax.set_ylabel(label, rotation=90)
plt.show()

fig_path = "global_subduction_%s_%.2fMa.pdf" % (attribute, reconstruction_time)

fig.savefig(fig_path)

print("figure saved: %s" % fig_path)

### Get the age of the subducting plate at these query points

Issue: We get a lot of nan value in ages
1. check the position of the sample points
2. plot the raster of oceanic plate ages

In [ ]:
help(age_grid_raster)

In [ ]:
fig1, ax1 = plt.subplots()
total_points_plotted = 0 # record the number of plotted points
for i in range(len(trench_ids)):
    trench_id = trench_ids[i]
    one_subduction_data_resampled = subduction_data_resampled_list[i]
    ages = age_grid_raster.interpolate(one_subduction_data_resampled.lon, one_subduction_data_resampled.lat)
    total_points_plotted += len(ages)
    ax1.plot(one_subduction_data_resampled.trench_velocity, ages, '*')

# configuration
ax1.set_xlabel("Trench Velocity Magnitude (cm/yr)")
ax1.set_ylabel("Age (Ma)")

print("Total plotted points: %d" % total_points_plotted)

fig2, ax2 = plt.subplots()
total_points_plotted = 0 # record the number of plotted points
all_ages = [] # record all the ages
all_trench_velocities = [] # record all the trench velocities
valid_age_value = 0 # record valid and non-valid values
non_valid_age_value = 0
for i in range(len(trench_ids)):
    trench_id = trench_ids[i]
    one_subduction_data_resampled = subduction_data_resampled_list[i]
    ages = age_grid_raster.interpolate(one_subduction_data_resampled.lon, one_subduction_data_resampled.lat, method="nearest")
    print("i = %d" % i)
    # print(type(one_subduction_data_resampled.lon))
    print("longitude:\n", one_subduction_data_resampled.lon) # debug
    print("latitude:\n", one_subduction_data_resampled.lat)
    print("ages: \n", ages)
    for i in range(len(ages)):
        age = ages[i]
        if not np.isnan(age):
            all_ages.append(age)
            all_trench_velocities.append(one_subduction_data_resampled.trench_velocity[i])
            valid_age_value += 1
        else:
            non_valid_age_value += 1
    # all_ages += ages
    # all_trench_velocities += one_subduction_data_resampled.trench_velocity.copy()
    total_points_plotted += len(ages)

print("Get %d valid age values and %d non-valid age values" % (valid_age_value, non_valid_age_value))
ax2.plot(range(valid_age_value), all_ages, '*')